In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import warnings

warnings.filterwarnings("ignore")

# Accessors for WSIData

The accessor is a concept that use attributes to extend the capabilities of a class.

There are three in-built accessors in the WSIData class:

- {py:class}`fetch <wsidata.FetchAccessor>`: Fetch information about the WSI
- {py:class}`iter <wsidata.IterAccessor>`: Iterate over the content of the WSI
- {py:class}`ds <wsidata.DatasetAccessor>`: Create deep learning datasets from the WSI

Here, we will load a WSI that have already been processed with tissue detection, tissue tiling, and feature extraction.

In your case, you can easily run these steps with `LazySlide` package.

In [ ]:
import pooch

slide = pooch.retrieve(
    "https://lazyslide.blob.core.windows.net/lazyslide-data/GTEX-1117F-0526.svs",
    fname="GTEX-1117F-0526.svs",
    known_hash="sha256:222ab7f2bb42dcd0bcfaccd910cb13be452b453499e6117ab553aa6cd60a135e",
)
_ = pooch.retrieve(
    "https://lazyslide.blob.core.windows.net/lazyslide-data/GTEX-1117F-0526.zarr.zip",
    fname="GTEX-1117F-0526.zarr.zip",
    known_hash="sha256:ef5f7e16599253c4bd7024b16730a48d80479181cb5d5135caef928c5e69b601",
    processor=pooch.Unzip(extract_dir="GTEX-1117F-0526.zarr"),
)

In [ ]:
from wsidata import open_wsi

wsi = open_wsi(slide)

In [ ]:
wsi

# Fetch accessor

Fetch accessor allows you to fetch essential information from WSIData.

## 1. Pyramids information

In [ ]:
wsi.fetch.pyramids()

## 2. Retrive the features as AnnData

In [ ]:
wsi.fetch.features_anndata("resnet50")

# Iter accessor

Like the name, the iter accessor will always return an iterator, and the iterator will always return data containers.

The data container usually implements a plot method for inspection.

## 1. Tissue contours

In [ ]:
d = next(wsi.iter.tissue_contours("tissues"))
d

It's also possible to visualize what's inside.

In [ ]:
d.plot()

You can use a for loop to iterate every tissue

In [ ]:
for d in wsi.iter.tissue_contours("tissues"):
    d.contour

## 2. Tissue images

Iterate through tissue images

In [ ]:
no_mask = next(wsi.iter.tissue_images("tissues"))
with_mask = next(wsi.iter.tissue_images("tissues", mask_bg=True))

In [ ]:
import matplotlib.pyplot as plt

_, (ax1, ax2) = plt.subplots(ncols=2)

no_mask.plot(ax=ax1)
with_mask.plot(ax=ax2)

## 3. Tile images

You can also iterate over all tile images

In [ ]:
d = next(wsi.iter.tile_images("tiles"))
d

You can include pathological annotations, this is useful to prepare dataset for training segmentation model.

In [ ]:
for d in wsi.iter.tile_images(
    "tiles", annot_key="annotations", annot_names="name", annot_labels={"sclerosis": 1}
):
    if len(d.annot_shapes) > 1:
        d.plot()
        break

## Dataset accessor

In [ ]:
dataset = wsi.ds.tile_images()

The dataset is a torch dataset that can be used to train a deep learning model. You can load it in the DataLoader and train the model.

In [ ]:
from torch.utils.data import DataLoader

dl = DataLoader(dataset, batch_size=36, shuffle=True)